# ⚔️ Week 4 Day 6: 全真模拟面试 (Mock Interview)

**今日主题**: **全真模拟面试 (Mock Interview Simulation)**。

我们将模拟一场真实的 AI 算法工程师面试。流程分为三个部分：

1.  **算法手撕 (Coding)**: 限时解决两道高频 Medium 题（综合考察本周知识点）。
2.  **项目拷打 (Project Deep Dive)**: 针对你的 `src` 代码架构进行提问，你需要准备好“话术”。
3.  **八股文突击 (Technical Q\&A)**: 考察 PyTorch 底层原理。

## 1 算法手撕 (Coding Round)

请尽量在 **30 分钟** 内完成这两道题。它们分别考察了 **双指针** 和 **字符串处理** 的综合能力。

### 1.1 三数之和 (3Sum) - Medium (面试出现率 Top 3)

  * **LeetCode 链接**: [15. 3Sum](../../LeetCode%20practice/1-50.ipynb)
  * **考察点**: 排序 + 双指针。它是 Day 1 "Two Sum" 的进阶版，也是面试官最爱出的题之一，因为边界条件（去重）很难写对。
  * **要求**: 给你一个整数数组 `nums`，判断是否存在三元组 `[nums[i], nums[j], nums[k]]` 满足 `i != j != k` 且和为 `0`。

In [ ]:
from typing import List

class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        nums.sort() # 1. 必须先排序
        res = []
        n = len(nums)
        
        for i in range(n):
            # 剪枝：如果第一个数已经大于0，后面不可能凑出0
            if nums[i] > 0: 
                break
            
            # 去重：对于第一个数，如果和前一个一样，跳过
            if i > 0 and nums[i] == nums[i-1]:
                continue
                
            # 双指针：在剩下的部分找 Two Sum = -nums[i]
            left, right = i + 1, n - 1
            while left < right:
                total = nums[i] + nums[left] + nums[right]
                
                if total == 0:
                    res.append([nums[i], nums[left], nums[right]])
                    
                    # 找到答案后，继续移动指针，并去重
                    while left < right and nums[left] == nums[left+1]:
                        left += 1
                    while left < right and nums[right] == nums[right-1]:
                        right -= 1
                    
                    left += 1
                    right -= 1
                    
                elif total < 0:
                    left += 1
                else:
                    right -= 1
                    
        return res

# --- 交互式测试 ---
solution = Solution()
print("=== 模拟面试题 1: 三数之和 ===")
raw_input = input("请输入一组数字 (例如 -1,0,1,2,-1,-4): ").strip()

if raw_input:
    nums = [int(x.strip()) for x in raw_input.split(',')]
    print(f"输入数组: {nums}")
    print(f"结果: {solution.threeSum(nums)}")
else:
    print("输入为空！")

### 1.2 翻转字符串里的单词 (Reverse Words in a String) - Medium

  * **LeetCode 链接**: [151. Reverse Words in a String](../../LeetCode%20practice/151-200.ipynb)
  * **考察点**: 字符串清洗 (`strip`), 列表翻转。
  * **要求**: 输入 `"  the sky  is blue  "`, 输出 `"blue is sky the"`。注意去除多余空格。

In [ ]:
class Solution():
    def reverseWords(self, s:str) -> str:
        new_s = s.split()
        left = 0
        right = len(new_s) - 1
        while left < right:
            new_s[left], new_s[right] = new_s[right], new_s[left]
            left += 1
            right -= 1
        return " ".join(new_s)
    
solition = Solution()
s_input = input("请输入字符串s：")
reveresd_s = solition.reverseWords(s_input)
print(reveresd_s)

## 2 项目拷打 (Project Defense)

假设我是面试官，我看着你的简历上写着 **"基于 PyTorch 构建模块化 CNN 图像分类系统"**。我会问你以下问题，请在 Notebook 中尝试用文字（MarkDown）回答：

**Q1: 为什么你的项目没有全部写在一个 Notebook 里，而是分成了 `src/models`, `src/engine.py`? 这样做有什么具体的工程好处？**

> *参考回答方向 (请用自己的话组织)*:
>
>   * **可复用性**: `SimpleCNN` 类定义在 `models.py` 里，可以在训练脚本调用，也可以在 `predict.py` 甚至 Web 服务脚本里调用。如果在 Notebook 里，想复用就得复制粘贴。
>   * **可维护性**: 把训练逻辑封装在 `engine.py` (train\_one\_epoch)，如果要修改 Loss 计算逻辑，只需要改这一个文件，不用去翻几十个 Notebook Cell。
>   * **团队协作**: Git 管理时，`.py` 文件的 diff 清晰可读，而 `.ipynb` 的 diff 是一场灾难。

**Q2: 在你的 `predict.py` 中，为什么输入图片前要加一个 `unsqueeze(0)`？**

> *参考回答方向*:
>
>   * PyTorch 的卷积层 (`nn.Conv2d`) 预期输入是 4 维张量：`[Batch_Size, Channels, Height, Width]`。
>   * 单张加载进来的图片通常是 `[C, H, W]` (例如 `[1, 28, 28]`)。
>   * `unsqueeze(0)` 实际上是在第 0 维增加了一个 Batch 维度，使其变成 `[1, 1, 28, 28]`，满足模型输入要求。

## 3 深度学习八股文 (Technical Q\&A)

这些问题不需要写代码，但需要你通过代码理解背后的逻辑。

**Q1: `optimizer.zero_grad()` 是做什么的？如果我把它忘了会发生什么？**

> *答案*:
>
>   * **作用**: 清空（归零）模型参数中累积的梯度 (`w.grad`)。
>   * **后果**: PyTorch 默认会**累加梯度** (Accumulate Gradients)。如果你不归零，第二次 `backward()` 算出的梯度会和第一次的加在一起，导致梯度越来越大，模型权重更新方向错误， Loss 震荡甚至发散。

**Q2: `CrossEntropyLoss` 里面包含了哪两个操作？**

> *答案*:
>
>   * 包含了 `LogSoftmax` 和 `NLLLoss` (Negative Log Likelihood Loss)。
>   * **陷阱**: 所以在定义模型 `SimpleCNN` 的最后一层全连接层时，**不需要**再加 `Softmax` 激活函数，输出原始 Logits 即可。如果加了 Softmax 再送入 CrossEntropy，等于做了一次多余且可能导致数值不稳定的计算。

### ✅ 任务与总结

1.  **实战**: 在 Notebook 中跑通两道算法题。
2.  **模拟**: 试着大声回答 Part 2 和 Part 3 的问题（就像对面坐着面试官）。
3.  **准备**:
      * Day 6 结束后，我们 Stage 1 的主要学习任务就结束了。
      * 明天 (Day 7) 是 **Week 4 总结与 Stage 2 预读日**。